In [24]:
import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# df = pd.read_csv('Data/mcts_trial_data.csv', sep='\t', encoding='utf-8')
df = pd.read_pickle('Data/mcts_trial_data.pkl')
cols = ['Board', 'Moves','Move_Probs','Reward']

print(df.head)
print(df.ndim, ' dimensions')
print(df.shape, ' shape')
print(df.size, ' size')

<bound method NDFrame.head of                                                 Board  \
0   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
1   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
2   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
3   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
5   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
6   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
7   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
8   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
9   [[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
10  [[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
11  [[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
12  [[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
13  [[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
14  [[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
15  [[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [...   
1

In [30]:
from board import Board

def convertToMatrix(test_str, K):
    # slicing strings
    temp = [test_str[idx: idx + K] for idx in range(0, len(test_str), K)]
 
    # conversion to list of characters
    res = [list(ele) for ele in temp]
 
    # printing result
    return res
 
 
row = 12
print(df.iloc[1])
x = df.iloc[row]['Board']
board = Board(1)
board.show_dots = False
board.board = x
print(board)
print(df.iloc[row]['Move_Probs'])
print(df.iloc[row]['Moves'])
print(len(df.iloc[row]['Move_Probs']), len(df.iloc[row]['Moves']))

Board         [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...
Moves         [[5, 8, 20, 1, 2, 0], [1, 4, 20, 7, 4, 0], [3,...
Move_Probs    [0.01, 0.0006666666666666666, 0.00066666666666...
Reward                                                        1
Name: 1, dtype: object
   A B C D E F G H I J K L M N
 1 □ □ □ □ □ □ ▣ ▣ ▣ □ □ □ □ □ 
 2 □ □ □ ▣ ▣ ▣ ▣ ▣ □ ▣ □ ▣ □ □ 
 3 □ □ ▣ ▣ ▣ ▣ ▣ ▣ ▣ □ □ □ ▣ □ 
 4 ▣ ▣ □ ▣ ▣ □ □ □ ▣ □ □ ▣ ▣ □ 
 5 ▣ ▣ ▣ □ ▣ □ □ □ □ ▣ ▣ □ ▣ □ 
 6 ▣ □ ▣ □ □ ▣ ▣ ▣ ▣ ▣ □ □ ▣ □ 
 7 ▣ □ ▣ □ ▣ ▣ □ □ ▣ ▣ ▣ ▣ ▣ □ 
 8 □ ▣ ▣ ▣ □ □ ▣ ▣ ▣ ▣ □ □ □ □ 
 9 □ □ ▣ □ □ □ □ ▣ ▣ ▣ ▣ □ ▣ □ 
10 □ □ □ ▣ ▣ ▣ ▣ ▣ ▣ ▣ ▣ ▣ ▣ ▣ 
11 □ □ □ ▣ ▣ □ □ ▣ ▣ □ ▣ ▣ ▣ □ 
12 □ □ ▣ ▣ ▣ ▣ ▣ ▣ □ ▣ ▣ ▣ ▣ □ 
13 □ □ □ □ □ ▣ ▣ ▣ □ ▣ ▣ □ □ □ 
14 □ □ □ □ □ □ □ □ ▣ ▣ ▣ □ □ □ 

[0.0006666666666666666, 0.0006666666666666666, 0.0006666666666666666, 0.0006666666666666666, 0.0006666666666666666, 0.0006666666666666666, 0.0006666666666666666, 0.0006666666666666666, 0.9946666666666667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0